# Tensors

From the [programmer's guide](https://www.tensorflow.org/programmers_guide/tensors):

> TensorFlow, as the name indicates, is a framework to define and run computations involving tensors. A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
m1 = [[1.0, 2.0],
      [3.0, 4.0]]
      
m2 = np.array([[1.0, 2.0],
              [3.0, 4.0]], dtype=np.float32)
               
m3 = tf.constant([[1.0, 2.0],
     [3.0, 4.0]])

In [ ]:
print(type(m1))
print(type(m2))
print(type(m3))

In [ ]:
t1 = tf.convert_to_tensor(m1, dtype=tf.float32)
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)
t3 = tf.convert_to_tensor(m3, dtype=tf.float32)

In [ ]:
print(type(t1))
print(type(t2))
print(type(t3))

Each tensor has a data type and a shape

In [ ]:
t = tf.constant([[1.0, 2.0, 3.0],
     [3.0, 4.0, 5.0]])
print(t.dtype)
print(t.get_shape())

In [ ]:
# In tensor flow we can think of the rank as the number of indices needed to
# specify an element.
my_image = tf.zeros([10, 299, 299, 3])  # batch x height x width x color
r = tf.rank(my_image)
with tf.Session() as sess:
    rank_val = sess.run([r])
    print(rank_val)

### Initialization

Tensor flow has some covenience methods for creating tensors initialized with zeros (tf.zero) and ones (tf.ones). Both take a shape argument.

In [ ]:
a = tf.ones([5,5]) * 0.5
print(a)

In [ ]:
with tf.Session() as sess:
    print(sess.run([a]))

### Creating Operators

Here we'll some examples of creating operators. Note that we define how operations behave, which is 
distinct from computing their value.

Refer to the TensorFlow documentation for more details on the operators offered by the library. As
an example, [here](https://www.tensorflow.org/api_guides/python/math_ops) are the math operators.

In [ ]:
# Negation operator
x = tf.constant([[1,2]])
negMatrix = tf.negative(x)
print(negMatrix)

### Executing Operators

From TensorFlow in Action:

> In general a session is an environment of a software system that describes how lines of code should be run.
In TensorFlow, a session sets up how the hardware (e.g. CPU, GPU) talk to each other.

The idea is you define your machine learning algorithm in a high level, declarative way, then bind it later to
its execution environment.

In [ ]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(negMatrix)
    
print(result)

Note you can run interactive sesssions as well, where you instantiate the session which is then implicitly
in scope for all subseqyent evals.

In [ ]:
sess = tf.InteractiveSession()
x = tf.constant([[1., 2.]])
negMatrix = tf.negative(x)
result = negMatrix.eval()
print(result)
sess.close()

### Variables

In TensorFLow, variables are used to store values that change over time

In [ ]:
sess = tf.InteractiveSession()

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13.]
spike = tf.Variable(False)
spike.initializer.run()

for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike,True)
        updater.eval()
    else:
        tf.assign(spike, False).eval()
        
    print("Spike",spike.eval())
    
sess.close()
            

### Variables Can Be Saved and Loaded

In [ ]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()
  dec_v2.op.run()
  # Save the variables to disk.
  save_path = saver.save(sess, "/tmp/model.ckpt")
  print("Model saved in path: %s" % save_path)

In [ ]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())